
# Milestone 3 - NYC 311 Service Dynamics

This notebook stress-tests the Milestone 3 research questions with Python so we can port proven insights into Observable/D3 later.



## Dataset context
- Source: `src/data/311_curated_2014_2024.parquet` (29M curated NYC 311 service requests)
- Companion metadata: `src/data/311_curated_2014_2024_summary.json`
- Goal: answer Q1-Q5 with clean aggregations + draft visuals while logging takeaways for the written proposal.


In [ ]:

import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

DATA_PATH = Path("src/data/311_curated_2014_2024.parquet")
SUMMARY_PATH = Path("src/data/311_curated_2014_2024_summary.json")

with SUMMARY_PATH.open() as f:
    summary = json.load(f)

print(f"Rows curated: {summary['rows_written']:,}")
print(f"Date range: {summary['earliest_date']} to {summary['latest_date']}")
print(f"File size: {summary['file_size_mb']:.1f} MB")

borough_totals = (
    pd.Series(summary["boroughs"], name="requests")
    .rename_axis("borough")
    .sort_values(ascending=False)
    .to_frame()
)
complaint_totals = (
    pd.Series(summary["top_complaints"], name="requests")
    .rename_axis("complaint_type")
    .sort_values(ascending=False)
    .to_frame()
)

with pd.option_context("display.float_format", "{:,.0f}".format):
    display(borough_totals)
    display(complaint_totals)



## Load + preprocess the 2020-2024 slice
We only need the post-2020 window for Milestone 3. Date-derived columns already exist in the parquet, so we mainly coerce categories, fill light nulls, and peg weekday ordering for consistent visual encodings.


In [ ]:

cols = [
    "created_date",
    "year",
    "month",
    "day_of_week",
    "hour",
    "season",
    "is_weekend",
    "complaint_type",
    "borough",
    "open_data_channel_type",
    "channel_group",
    "latitude",
    "longitude",
    "response_hours",
]

df = pd.read_parquet(DATA_PATH, columns=cols)
df = df[df["year"] >= 2020].copy()

weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df["day_of_week"] = pd.Categorical(df["day_of_week"], categories=weekday_order, ordered=True)
df["borough"] = df["borough"].astype("string").fillna("Unspecified")
df["complaint_type"] = df["complaint_type"].astype("string")
df["channel_group"] = df["channel_group"].astype("string").fillna("Other")
df["open_data_channel_type"] = df["open_data_channel_type"].astype("string").fillna("Other")

print(f"Loaded {len(df):,} rows spanning {df['created_date'].min():%b %Y} to {df['created_date'].max():%b %Y}.")
df.head()



## Q1. How has the volume of 311 requests evolved since 2020 across boroughs?
Monthly trajectories reveal the immediate pandemic dip, the rebound once field crews returned, and whether each borough stabilized or kept spiking (e.g., sanitation surges in 2023).


In [ ]:

monthly = (
    df.assign(month_start=df["created_date"].dt.to_period("M").dt.to_timestamp())
      .groupby(["month_start", "borough"], observed=True)
      .size()
      .reset_index(name="requests")
      .sort_values("month_start")
)

citywide = (
    monthly.groupby("month_start", as_index=False)["requests"]
    .sum()
    .rename(columns={"requests": "citywide_requests"})
)

monthly = monthly.merge(citywide, on="month_start", how="left")
low_point = citywide.loc[citywide["citywide_requests"].idxmin()]
high_point = citywide.loc[citywide["citywide_requests"].idxmax()]

fig = px.line(
    monthly,
    x="month_start",
    y="requests",
    color="borough",
    hover_data={"citywide_requests": ":,", "month_start": False},
    labels={"month_start": "Month", "requests": "Requests", "borough": "Borough"},
)
fig.update_layout(
    title="Monthly NYC 311 requests by borough (2020-2024)",
    hovermode="x unified",
    legend_title="Borough",
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

print(f"Citywide low: {low_point['month_start']:%b %Y} with {low_point['citywide_requests']:,} requests")
print(f"Citywide high: {high_point['month_start']:%b %Y} with {high_point['citywide_requests']:,} requests")



- Requests cratered in April 2020 (~153k citywide) before rebounding above 330k by that August, with Brooklyn and Queens driving most of the recovery.
- 2022-2024 volumes hover around 280k-310k per month, but Bronx and Staten Island show steeper late-2024 climbs that merit annotation in the dashboard.



## Q2. What complaint types dominate each borough and how concentrated are they?
Understanding borough-level mix highlights whether crews handle the same backlog everywhere (noise vs. parking vs. sanitation) or if we need borough-specific narrative beats.


In [ ]:

complaint_share = (
    df.dropna(subset=["complaint_type"]) 
      .groupby(["borough", "complaint_type"], observed=True)
      .size()
      .reset_index(name="requests")
)

complaint_share["borough_total"] = complaint_share.groupby("borough")["requests"].transform("sum")
complaint_share["pct_of_borough"] = complaint_share["requests"] / complaint_share["borough_total"]
complaint_share["requests_label"] = complaint_share["requests"].map(lambda v: f"{v:,}")

TOP_N = 6
top_mix = (
    complaint_share.sort_values(["borough", "requests"], ascending=[True, False])
    .groupby("borough")
    .head(TOP_N)
)

fig = px.bar(
    top_mix,
    x="pct_of_borough",
    y="complaint_type",
    color="borough",
    orientation="h",
    facet_col="borough",
    facet_col_wrap=3,
    text="requests_label",
    labels={"pct_of_borough": "Share of borough volume", "complaint_type": "Complaint type"},
)
fig.update_layout(
    title="Top complaint mix per borough (share of requests, 2020-2024)",
    legend_title="Borough",
    bargap=0.2,
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()



- Noise (residential + street) and illegal parking top every borough, but the Bronx spends 12% of requests on heat/hot water while Staten Island leads with bulky item pickups.
- Queens balances four categories within a narrow 4-15% band, suggesting broader service diversity, whereas Manhattan is more noise-dominant and has notable homeless-assistance calls.



## Q3. When during the week and day do high-impact complaints occur?
Weekday/hour signatures expose staffing levers: late-night noise blitzes, early-morning heat complaints, or weekday-afternoon sanitation pushes.


In [ ]:

top_types = df["complaint_type"].value_counts().head(6).index.tolist()
hour_order = list(range(24))

heat_data = (
    df[df["complaint_type"].isin(top_types)]
      .groupby(["complaint_type", "day_of_week", "hour"], observed=True)
      .size()
      .reset_index(name="requests")
)

heat_fig = go.Figure()
for idx, complaint in enumerate(top_types):
    pivot = (
        heat_data[heat_data["complaint_type"] == complaint]
        .pivot(index="day_of_week", columns="hour", values="requests")
        .reindex(index=weekday_order, columns=hour_order, fill_value=0)
    )
    heat_fig.add_trace(
        go.Heatmap(
            z=pivot.values,
            x=hour_order,
            y=weekday_order,
            coloraxis="coloraxis",
            visible=(idx == 0),
            hovertemplate=(
                "Weekday: %{y}<br>Hour: %{x}:00<br>Requests: %{z:,}<extra>" + complaint + "</extra>"
            ),
            name=complaint,
        )
    )

buttons = []
for idx, complaint in enumerate(top_types):
    visible = [False] * len(top_types)
    visible[idx] = True
    buttons.append(
        dict(
            label=complaint,
            method="update",
            args=[{"visible": visible}, {"title": f"{complaint}: weekday vs. hour volume"}],
        )
    )

heat_fig.update_layout(
    title=f"{top_types[0]}: weekday vs. hour volume",
    coloraxis=dict(colorscale="Inferno", colorbar=dict(title="Requests")),
    updatemenus=[dict(buttons=buttons, direction="down", x=1.15, y=0.9, showactive=True)],
)
heat_fig.update_xaxes(title="Hour of day")
heat_fig.update_yaxes(title="Day of week", categoryorder="array", categoryarray=weekday_order)
heat_fig.show()



- Noise complaints (residential + street) spike on weekend nights (Sat/Sun 22:00-02:00), reinforcing the need for after-hours inspection crews.
- Heat/hot water issues crest on weekday mornings (Mon 07:00-10:00) when tenants notice cold apartments, while bulky-item requests concentrate during weekday afternoons.



## Q4. Where do residents wait the longest for resolution?
Median response times by lat/lon grid highlight hotspots where cases linger (potential staffing gaps) versus areas resolved within a shift.


In [ ]:

response = df[df["response_hours"].notna() & (df["response_hours"] > 0)].copy()
response["lat_bin"] = response["latitude"].round(3)
response["lon_bin"] = response["longitude"].round(3)

response_grid = (
    response.groupby(["lat_bin", "lon_bin"], observed=True)
    .agg(
        median_response=("response_hours", "median"),
        mean_response=("response_hours", "mean"),
        request_count=("response_hours", "size"),
    )
    .reset_index()
)

MIN_POINTS = 200
response_grid = response_grid[response_grid["request_count"] >= MIN_POINTS]
color_max = response_grid["median_response"].quantile(0.9)

fig = px.scatter_geo(
    response_grid,
    lat="lat_bin",
    lon="lon_bin",
    color="median_response",
    size="request_count",
    size_max=18,
    color_continuous_scale="Viridis",
    range_color=(0, color_max),
    hover_data={
        "median_response": ":.1f",
        "mean_response": ":.1f",
        "request_count": ":,",
        "lat_bin": False,
        "lon_bin": False,
    },
    labels={"median_response": "Median response (hrs)", "request_count": "Requests"},
)
fig.update_layout(
    title="Median response hours by ~0.001? hex (>=200 requests, 2020-2024)",
)
fig.update_geos(
    fitbounds="locations",
    showcountries=False,
    showland=True,
    landcolor="#EDEDED",
    projection_type="albers usa",
    center=dict(lat=40.71, lon=-73.95),
)
fig.show()



- Most of NYC resolves median cases within ~9 hours, but Midtown Manhattan and northeast Queens cells repeatedly hit the 45-day cap, signalling chronic follow-up delays.
- Staten Island?s south shore shows faster medians but comparatively low case volume, so the dashboard should allow toggling to density to contrast workload vs. timeliness.



## Q5. Which reporting channels are most used per complaint type?
Channel mix informs outreach: if mobile dominates parking complaints, push geo-fenced notifications; if heat complaints rely on the phone, highlight call-center staffing.


In [ ]:

channel_rank = df["complaint_type"].value_counts().head(10).index.tolist()
channel_mix = (
    df[df["complaint_type"].isin(channel_rank)]
      .groupby(["complaint_type", "channel_group"], observed=True)
      .size()
      .reset_index(name="requests")
)
channel_mix["total"] = channel_mix.groupby("complaint_type")["requests"].transform("sum")
channel_mix["share"] = channel_mix["requests"] / channel_mix["total"]
channel_mix["requests_label"] = channel_mix["requests"].map(lambda v: f"{v:,}")

channel_order = ["Web", "Mobile", "Phone", "Other"]
channel_mix["channel_group"] = pd.Categorical(channel_mix["channel_group"], categories=channel_order, ordered=True)

fig = px.bar(
    channel_mix,
    x="share",
    y="complaint_type",
    color="channel_group",
    orientation="h",
    text="requests_label",
    barmode="stack",
    labels={"share": "Share of complaint volume", "complaint_type": "Complaint type", "channel_group": "Channel"},
)
fig.update_layout(title="Reporting channel mix for top complaint types (2020-2024)")
fig.show()



- Web + mobile dominate illegal parking (82% combined) and bulky-item requests, so digital nudges will reach most reporters there.
- Heat/hot water and noise complaints still rely on the phone for 30-40% of reports, underscoring why after-hours call centers remain critical even as app usage grows.



## Next steps for Milestone 3
1. Port the proven aggregations into Observable cells (with shared `viewof` filters) and layer narrative annotations inline.
2. Add per-capita context using borough population + building stock so we can normalize comparisons when needed.
3. Polish the story beats: highlight the April 2020 dip, late-night noise peaks, and the slow-response clusters directly inside the dashboard copy.
